In [3]:
from pymongo import MongoClient

client = MongoClient()
db = client['SportsAnalytics']
colStat = db['CollegeStatistics']
draft = db['DraftProspects']

In [7]:
import pandas as pd
import numpy as np

In [14]:
college_statistics = pd.read_csv('archive/college_statistics.csv')
draft_prospects = pd.read_csv('archive/nfl_draft_prospects.csv')

In [11]:
college_statistics_pivot = college_statistics.pivot_table(index=['player_id', 'alt_player_id', 'player_name', 'pos_abbr', 'school', 'school_abbr', 'school_primary_color', 'school_alt_color', 'season', 'active', 'all_star'],
                            columns='statistic',
                            values='value').reset_index()

In [12]:
idx = college_statistics_pivot.groupby('player_id')['season'].idxmax()
college_statistics_filtered = college_statistics_pivot.loc[idx]

In [15]:
threshold_value = 2014
column_to_check = 'draft_year'  # Adjust this column as needed
draft_prospects_filter = draft_prospects[(draft_prospects[column_to_check] >= threshold_value) & (draft_prospects[column_to_check] < 2021)]

In [16]:
merged_df = pd.merge(college_statistics_filtered, draft_prospects_filter, on='player_id', how='left')

----
This is the merged_df we are going to use to build the model

In [17]:
merged_df.head()

,player_id,alt_player_id,player_name_x,pos_abbr_x,school_x,school_abbr_x,school_primary_color,school_alt_color,season,active,...,team,team_abbr,team_logo_espn,guid,weight,height,pos_rk,ovr_rk,grade,player_image
0,368,3924331,Justin Smith,DE,Missouri,MIZ,#000000,#000000,2016,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4019,4239824,Bryan Thomas,DE,UAB,UAB,#003b28,#ffc845,2018,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4542,4240031,Derrick Brooks,LB,Florida State,FSU,#782F40,#ceb888,2020,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4559,4240091,Joe Johnson,DE,Louisville,LOU,#ad000a,#cccccc,2020,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14420,3915189,Royce Smith,OG,Georgia,UGA,#CC0000,#000000,2017,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
columns_to_drop = [
    'player_id', 'player_name_x', 'alt_player_id', 'school_abbr_x', 
    'school_primary_color', 'school_alt_color', 'pos_abbr_y', 'school_y', 
    'school_abbr_y', 'pick', 'overall', 'team_logo_espn', 'player_image', 
    'player_name_y', 'traded', 'trade_note', 'link', 'team_abbr', 'guid'
]
df = pd.get_dummies(merged_df, columns=['school_x', 'pos_abbr_x', 'school_name', 'position', 'team'])

# Create a binary column for drafted/not drafted
df['drafted'] = df['round'].notna().astype(int)
#df['drafted'] = ~df['round'].isna().astype(int)
print(df['round'].value_counts())
print(df['drafted'].value_counts())

# Now define the target variable
y = df['drafted']

# Drop the unnecessary columns, including those you created dummies from if they're no longer needed
df = df.drop(columns_to_drop + ['round'], axis=1, errors='ignore')

# Fill NaN values with 0
df = df.fillna(0)

# Your features are the remaining columns
X = df.drop(['drafted'], axis=1)

round
1.0    90
4.0    90
2.0    88
3.0    85
6.0    77
7.0    76
5.0    75
Name: count, dtype: int64
drafted
1    581
0    545
Name: count, dtype: int64


In [35]:
y.unique()

array([0, 1])

In [36]:
X

,season,active,all_star,Assist Tackles,Completion Percentage,Completions,Extra Points Made,FGM 1-19 yards,FGM 20-29 yards,FGM 30-39 yards,...,team_New York Jets,team_Oakland Raiders,team_Philadelphia Eagles,team_Pittsburgh Steelers,team_San Francisco 49ers,team_Seattle Seahawks,team_Tampa Bay Buccaneers,team_Tennessee Titans,team_Washington,team_Washington Redskins
0,2016,True,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
1,2018,True,False,18.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
2,2020,True,False,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
3,2020,True,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
4,2017,True,False,52.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1121,2020,True,False,6.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
1122,2020,True,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
1123,2020,True,False,0.0,0.0,0.0,12.0,0.0,1.0,0.0,...,False,False,False,False,False,False,False,False,False,False
1124,2020,True,False,12.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False


In [37]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

c:\Users\prgol\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

In [38]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 87  15]
 [ 16 108]]
              precision    recall  f1-score   support

           0       0.84      0.85      0.85       102
           1       0.88      0.87      0.87       124

    accuracy                           0.86       226
   macro avg       0.86      0.86      0.86       226
weighted avg       0.86      0.86      0.86       226



In [39]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

# Print metrics
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.8628318584070797
Precision: 0.8613544873312811
Recall: 0.861954459203036
F1 Score: 0.8616372074651921


Let's try with grid search

In [44]:
from sklearn.model_selection import GridSearchCV

model = LogisticRegression(solver='saga')
parameters = {'C':[0.001,0.01,0.1,1,10,100,1000]}
grid = GridSearchCV(model,parameters,cv=5)

In [46]:
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings('ignore',category=ConvergenceWarning)
grid.fit(X_train, y_train)
print(grid.best_params_)

{'C': 0.001}


In [47]:
print(grid.best_score_)

0.8655555555555555


In [48]:
y_pred = grid.predict(X_test)

In [49]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

# Print metrics
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.8761061946902655
Precision: 0.8746059268600253
Recall: 0.8757906388361796
F1 Score: 0.8751282658457653


In [50]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 89  13]
 [ 15 109]]
              precision    recall  f1-score   support

           0       0.86      0.87      0.86       102
           1       0.89      0.88      0.89       124

    accuracy                           0.88       226
   macro avg       0.87      0.88      0.88       226
weighted avg       0.88      0.88      0.88       226

